# 1. Start

In [1]:
import warnings
warnings.filterwarnings("ignore")


# from google_auth_oauthlib.flow import Flow
# from IPython.display import Javascript
import matplotlib.pyplot as plt
import json
import requests

import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
# import ipywidgets as widgets
import statsmodels.api as sm
import statsmodels
from statsmodels.tsa.statespace.sarimax import SARIMAX
import itertools
from datetime import datetime, timedelta
warnings.filterwarnings("ignore")

from bokeh.plotting import figure, show
from bokeh.io import output_notebook, output_file
from bokeh.models import DaysTicker, FuncTickFormatter, ColumnDataSource, LabelSet, NumeralTickFormatter, DataTable, TableColumn, Div, HoverTool, Legend
from bokeh.layouts import gridplot, layout
from bokeh.transform import dodge
from bokeh.palettes import cividis
from bokeh.models.glyphs import Scatter

from sklearn.cluster import KMeans

from statsmodels.tsa.stattools import adfuller, kpss, acf, pacf, grangercausalitytests

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from oauth2client import client
from oauth2client import file
from oauth2client import tools
from google.oauth2 import service_account
from apiclient.discovery import build
import httplib2

import numpy as np
from bokeh.io import show, output_file
from bokeh.plotting import figure

output_notebook()



Loading BokehJS ...

# 2. Data extraction

In [10]:
start_df = (datetime.today() - timedelta(365*2)).strftime('%Y-%m-%d')
end_dt = datetime.today().strftime('%Y-%m-%d')

# read clients csv
acc = pd.read_csv('clients.csv')

#Sessions
df = pd.DataFrame(index = pd.date_range(freq = '1d', start = start_df, end = end_dt))
# GoalCompletions
df1 = pd.DataFrame(index = pd.date_range(freq = '1d', start = start_df, end = end_dt))

for n in range(len(acc)):
  try:
    r = requests.get('https://www.googleapis.com/analytics/v3/data/ga?ids=ga%3A' + str(acc['GA View Id'][n]) + '&start-date=' + start_df + '&end-date=' + end_dt + '&metrics=ga%3Asessions%2Cga%3AgoalCompletionsAll&dimensions=ga%3Adate&filters=ga%3Asource%3D%3Dgoogle%3Bga%3Amedium%3D%3Dorganic&access_token=' + tokens.get('access_token')) # make the request # removing segmet temporarily &segment=gaid::PCfSbG2jQ5i1K2dIn82OoQ
    data = r.json()

    temp_df = pd.DataFrame(data['rows'])[[1]]
    df[acc['Website Name'][n]] = temp_df[[1]].values
    
    temp_df1 = pd.DataFrame(data['rows'])[[2]]
    df1[acc['Website Name'][n]] = temp_df[[2]].values
  except: KeyError

def norm(df):
  # df = df.astype('int').diff().dropna() # differencing the data since it's not stationary; added 3/8
  df_scaled_seasonal = pd.DataFrame(index = pd.date_range(freq = '1d', start = start_df, end = end_dt))
  df_scaled_trend = pd.DataFrame(index = pd.date_range(freq = '1d', start = start_df, end = end_dt))
  for i in range(df.shape[1]):
    decomposition = sm.tsa.seasonal_decompose(pd.DataFrame(df.iloc[:, i].astype('int')), model='additive')
    min_max_scaler = preprocessing.MinMaxScaler()

    scaled_x_seasonal = min_max_scaler.fit_transform(decomposition.seasonal.values.reshape(-1, 1))
    df_scaled_seasonal[df.iloc[:, i].name] = scaled_x_seasonal

    scaled_x_trend = min_max_scaler.fit_transform(decomposition.trend.values.reshape(-1, 1))
    df_scaled_trend[df.iloc[:, i].name] = scaled_x_trend
  return df_scaled_seasonal, df_scaled_trend


def calc_trends(df, df_scaled_seasonal, df_scaled_trend):
  # df_scaled_seasonal, df_scaled_trend = norm(df)
  col0 = []
  col1 = []
  col2 = []

  for i in range(df_scaled_seasonal.shape[1]):
    for j in range(df_scaled_seasonal.shape[1]):
      col0.append(df_scaled_seasonal.iloc[:,i].name)
      col1.append(df_scaled_seasonal.iloc[:,j].name)
      col2.append(sum(abs(df_scaled_seasonal.iloc[:,i] - df_scaled_seasonal.iloc[:,j])))

  col3 = []
  col4 = []
  col5 = []
  col6 = []
  col7 = []
  col8 = []
  col9 = []
  col10 = []

  df_scaled_trend.dropna(inplace=True)
  for i in range(df_scaled_trend.shape[1]):
    for j in range(df_scaled_trend.shape[1]):
      col3.append(df_scaled_trend.iloc[:,i].name)
      col4.append(df_scaled_trend.iloc[:,j].name)
      col5.append(sum(abs(df_scaled_trend.iloc[:,i] - df_scaled_trend.iloc[:,j])))
      col6.append(sum(abs(df_scaled_trend.iloc[-31:,i] - df_scaled_trend.iloc[-31:,j])))
      res = grangercausalitytests(df_scaled_trend.iloc[:, [i, j]], maxlag = 2, verbose = False)
      col7.append(res[1][0]['lrtest'][1]) # p-value of 1st lag likelihood ratio test
      col8.append(res[2][0]['lrtest'][1]) # p-value of 2nd lag likelihood ratio test
      res2 = grangercausalitytests(df_scaled_trend.iloc[-31:, [i, j]], maxlag = 2, verbose = False)
      col9.append(res2[1][0]['lrtest'][1]) # p-value of 1st lag likelihood ratio test
      col10.append(res2[2][0]['lrtest'][1]) # p-value of 2nd lag likelihood ratio test


  df_x = pd.DataFrame(zip(col0, col1, col2), columns=['Client_A', 'Client_B', 'Seasonal_score'])
  df_y = pd.DataFrame(zip(col3, col4, col5, col6, col7, col8, col9, col10), columns=['Client_A', 'Client_B', 'Trend_score', 'Trend_score_1mo', 'LR_p', 'LR_p2', 'LR_p_1mo', 'LR_p2_1mo'])
  df_merged = df_x.merge(df_y, how = 'inner', on=['Client_A', 'Client_B'])

  df_merged = df_merged.loc[df_merged['Client_A'] != df_merged['Client_B']]

  # df_merged['Sim_seasonal'] = [(1 - i / 40) * 0.15 if i <= 40 else 0 for i in df_merged['Seasonal_score']]
  # df_merged['Sim_trend'] = [(1 - i / 80) * 0.10 if i <= 80 else 0 for i in df_merged['Trend_score']]
  # df_merged['Sim_trend_1mo'] = [(1 - i / 20) * 0.15 if i <= 20 else 0 for i in df_merged['Trend_score_1mo']]
  # df_merged['Sim_LR_p'] = [(1 - i / 0.05) * 0.15 if i <= 0.05 else 0 for i in df_merged['LR_p']]
  # df_merged['Sim_LR_p2'] = [(1 - i / 0.05) * 0.15 if i <= 0.05 else 0 for i in df_merged['LR_p2']]
  # df_merged['Sim_LR_p_1mo'] = [(1 - i / 0.05) * 0.05 if i <= 0.05 else 0 for i in df_merged['LR_p_1mo']]     #latest added signal, maybe adjust weight
  # df_merged['Sim_LR_p2_1mo'] = [(1 - i / 0.05) * 0.05 if i <= 0.05 else 0 for i in df_merged['LR_p2_1mo']]   #latest added signal, maybe adjust weight

  # df_merged['Sim_tot'] = df_merged['Sim_seasonal'] + df_merged['Sim_trend'] + df_merged['Sim_trend_1mo'] + df_merged['Sim_LR_p'] + df_merged['Sim_LR_p2'] + df_merged['Sim_LR_p_1mo'] + df_merged['Sim_LR_p2_1mo']
  
  return df_merged




df_scaled_seasonal, df_scaled_trend = norm(df)
df_merged = calc_trends(df, df_scaled_seasonal, df_scaled_trend)

SCOPES = ['https://www.googleapis.com/auth/webmasters.readonly']
# hardcoding credentials to avoid having to authenticate to a google drive file
SERVICE_ACCOUNT_FILE = 'service_account_file.json'


credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

webmasters_service = build('searchconsole', 'v1', credentials = credentials)


start_df0 = (datetime.today() - timedelta(365)).strftime('%Y-%m-%d')
end_dt0 = (datetime.today() - timedelta(3)).strftime('%Y-%m-%d')
df1 = pd.DataFrame(index = pd.date_range(freq = '1d', start = start_df0, end = end_dt0))
df2 = pd.DataFrame(index = pd.date_range(freq = '1d', start = start_df0, end = end_dt0))
df3 = pd.DataFrame(index = pd.date_range(freq = '1d', start = start_df0, end = end_dt0))

# legacy api call
for n in range(len(acc)):
  try:
    r1 = webmasters_service.searchanalytics().query(
      siteUrl = acc['URL'][n] , body = {
                                      'startDate': start_df0,
                                      'endDate': end_dt0,
                                      'dimensions': ['date'],
                                      'rowLimit': 25000
                                      }
                                                ).execute()
    temp_df1 = pd.DataFrame(r1['rows'])
    temp_df1.columns = ['Date', 'Clicks', 'Impressions', 'CTR', 'Position']

    for i in range(len(temp_df1['Date'])):
      temp_df1['Date'][i] = temp_df1['Date'][i][0]

    temp_df1 = temp_df1.iloc[:, [0, -1]] #Position
    df1[acc['Website Name'][n]] = temp_df1.iloc[:, 1].values

    temp_df2 = pd.DataFrame(r1['rows'])
    temp_df2.columns = ['Date', 'Clicks', 'Impressions', 'CTR', 'Position']

    for i in range(len(temp_df2['Date'])):
      temp_df2['Date'][i] = temp_df2['Date'][i][0]

    temp_df2 = temp_df2.iloc[:, [0, 1]] #Clicks
    df2[acc['Website Name'][n]] = temp_df2.iloc[:, 1].values

    temp_df3 = pd.DataFrame(r1['rows'])
    temp_df3.columns = ['Date', 'Clicks', 'Impressions', 'CTR', 'Position']

    for i in range(len(temp_df3['Date'])):
      temp_df3['Date'][i] = temp_df3['Date'][i][0]

    temp_df3 = temp_df3.iloc[:, [0, 2]] #Impressions
    df3[acc['Website Name'][n]] = temp_df3.iloc[:, 1].values
  except: requests.HTTPError

# Function definitions
def data_transform(df, start_date, end_date):
    col0 = []
    col1 = []
    col2 = []


    for s in range(df.shape[1]):
      col0.append(df.iloc[:,s].name)
      # period_len = min(len(df.iloc[:,s].loc[:start_date,]), len(df.iloc[:,s].loc[start_date:,])) #legacy code before adding end_date
      period_len = len(df.iloc[:,s].loc[start_date:end_date,])
      before_df = df.iloc[:,s].loc[:(pd.to_datetime(start_date) - timedelta(1)).strftime('%Y-%m-%d'),].iloc[-period_len+1:,]
      after_df = df.iloc[:,s].loc[(pd.to_datetime(start_date) + timedelta(1)).strftime('%Y-%m-%d'):,].iloc[:period_len,]
      col1.append(np.median(before_df.astype('int')))  # changed aggregation from mean to median 2021-08-17
      col2.append(np.median(after_df.astype('int')))   # changed aggregation from mean to median 2021-08-17
      # df.iloc[:,s].loc[:start_date,]

    df_comp = pd.DataFrame(zip(col0, col1, col2), columns=['Client', 'Median before', 'Median after'])
    df_comp['Difference'] = df_comp['Median after'] / df_comp['Median before'] - 1 
    df_comp

    # Error handling when division by zero or missing data for one of the periods for a client
    df_comp.loc[~np.isfinite(df_comp['Difference']), 'Difference'] = np.nan
    df_comp = df_comp.loc[-df_comp['Difference'].isnull()]


    col3 = []
    col4 = []
    col5 = []

    for s in range(df_scaled_trend.shape[1]):
      col3.append(df_scaled_trend.iloc[:,s].name)
      
      # period_len = min(len(df_scaled_trend.iloc[:,s].loc[:start_date,]), len(df_scaled_trend.iloc[:,s].loc[start_date:,])) #legacy code before adding end_date
      eriod_len = len(df_scaled_trend.iloc[:,s].loc[start_date:end_date,])
      before_df = df_scaled_trend.iloc[:,s].loc[:(pd.to_datetime(start_date) - timedelta(1)).strftime('%Y-%m-%d'),].iloc[-period_len+1:,].astype('float64')
      after_df = df_scaled_trend.iloc[:,s].loc[(pd.to_datetime(start_date) + timedelta(1)).strftime('%Y-%m-%d'):,].iloc[:period_len,].astype('float64')

      X_before = (before_df.index -  before_df.index[0]).days.values.reshape(-1, 1)
      Y_before = before_df.values
      LIN_before = LinearRegression()
      LIN_before.fit(X_before, Y_before)

      X_after = (after_df.index -  after_df.index[0]).days.values.reshape(-1, 1)
      Y_after = after_df.values
      LIN_after = LinearRegression()
      LIN_after.fit(X_after, Y_after)

      col4.append(LIN_before.coef_[0])
      col5.append(LIN_after.coef_[0])
      # df.iloc[:,s].loc[:start_date,]

    df_comp_slope = pd.DataFrame(zip(col3, col4, col5), columns=['Client', 'LR Slope before', 'LR Slope after'])
    df_comp_slope['Difference'] = df_comp_slope['LR Slope after'] / df_comp_slope['LR Slope before'] - 1
    return df_comp, df_comp_slope



def plot_hist(df_comp, n_bins, client):
    hist, edges = np.histogram(df_comp['Difference'], density=False, bins=n_bins)

    p0 = figure(height = 300, width = 700)
    p0.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="white", fill_color = '#00b5e2')
    p0.title.text = "Median change"

    if client != 'None':
      selected_client_bin = df_comp[df_comp['Client'] == client]['Difference'].values[0]
      glyph = Scatter(x=selected_client_bin, y=1, marker="square", size = 10, fill_color = 'orange')
      p0.add_glyph(glyph)

    outliers = df_comp.loc[(df_comp['Difference'] < - 3*df_comp['Difference'].std()) | (df_comp['Difference'] > 3*df_comp['Difference'].std())]
    outliers.iloc[:,1] = outliers.iloc[:,1].map("{:.2f}".format)
    outliers.iloc[:,2] = outliers.iloc[:,2].map("{:.2f}".format)
    outliers['Difference'] = outliers['Difference'].map("{:.2%}".format)

    p1 = DataTable(columns = [TableColumn(field=col, title=col) for col in outliers.columns], source = ColumnDataSource(outliers), height = 300, width = 450)
    p2 = DataTable(columns = [TableColumn(field=col, title=col) for col in df_comp.columns], source = ColumnDataSource(df_comp), height = 500, width = 750)

    p = gridplot([[p0, p1], [p2]])
    output_file("hist.html")
    show(p)

def plot_grid(df_comp1, df_comp2, df_comp3, n_bins, client):
    hist1, edges1 = np.histogram(df_comp1['Difference'], density=False, bins=n_bins)
    p1 = figure(height = 300, width = 700)
    p1.quad(top=hist1, bottom=0, left=edges1[:-1], right=edges1[1:], line_color="white", fill_color = '#00b5e2')
    p1.title.text = "Position - median change"

    if client != 'None':
      selected_client_bin1 = df_comp1[df_comp1['Client'] == client]['Difference'].values[0]
      glyph1 = Scatter(x=selected_client_bin1, y=1, marker="square", size = 10, fill_color = 'orange')
      p1.add_glyph(glyph1)


    hist2, edges2 = np.histogram(df_comp2['Difference'], density=False, bins=n_bins)
    p2 = figure(height = 300, width = 700)
    p2.quad(top=hist2, bottom=0, left=edges2[:-1], right=edges2[1:], line_color="white", fill_color = '#00b5e2')
    p2.title.text = "Clicks - median change"

    if client != 'None':
      selected_client_bin2 = df_comp2[df_comp2['Client'] == client]['Difference'].values[0]
      glyph2 = Scatter(x=selected_client_bin2, y=1, marker="square", size = 10, fill_color = 'orange')
      p2.add_glyph(glyph2)


    hist3, edges3 = np.histogram(df_comp3['Difference'], density=False, bins=n_bins)
    p3 = figure(height = 300, width = 700)
    p3.quad(top=hist3, bottom=0, left=edges3[:-1], right=edges3[1:], line_color="white", fill_color = '#00b5e2')
    p3.title.text = "Impressions - median change"

    if client != 'None':
      selected_client_bin3 = df_comp3[df_comp3['Client'] == client]['Difference'].values[0]
      glyph3 = Scatter(x=selected_client_bin3, y=1, marker="square", size = 10, fill_color = 'orange')
      p3.add_glyph(glyph3)


    outliers1 = df_comp1.loc[(df_comp1['Difference'] < - 3*df_comp1['Difference'].std()) | (df_comp1['Difference'] > 3*df_comp1['Difference'].std())]
    outliers1.iloc[:,1] = outliers1.iloc[:,1].map("{:.2f}".format)
    outliers1.iloc[:,2] = outliers1.iloc[:,2].map("{:.2f}".format)
    outliers1['Difference'] = outliers1['Difference'].map("{:.2%}".format)
    p1_1 = DataTable(columns = [TableColumn(field=col, title=col) for col in outliers1.columns], source = ColumnDataSource(outliers1), height = 300, width = 450)

    outliers2 = df_comp2.loc[(df_comp2['Difference'] < - 3*df_comp2['Difference'].std()) | (df_comp2['Difference'] > 3*df_comp2['Difference'].std())]
    outliers2.iloc[:,1] = outliers2.iloc[:,1].map("{:.2f}".format)
    outliers2.iloc[:,2] = outliers2.iloc[:,2].map("{:.2f}".format)
    outliers2['Difference'] = outliers2['Difference'].map("{:.2%}".format)
    p2_1 = DataTable(columns = [TableColumn(field=col, title=col) for col in outliers2.columns], source = ColumnDataSource(outliers2), height = 300, width = 450)

    outliers3 = df_comp3.loc[(df_comp3['Difference'] < - 3*df_comp3['Difference'].std()) | (df_comp3['Difference'] > 3*df_comp3['Difference'].std())]
    outliers3.iloc[:,1] = outliers3.iloc[:,1].map("{:.2f}".format)
    outliers3.iloc[:,2] = outliers3.iloc[:,2].map("{:.2f}".format)
    outliers3['Difference'] = outliers3['Difference'].map("{:.2%}".format)
    p3_1 = DataTable(columns = [TableColumn(field=col, title=col) for col in outliers3.columns], source = ColumnDataSource(outliers3), height = 300, width = 450)

    p = gridplot([[p1, p1_1], [p2, p2_1], [p3, p3_1]])
    # p = gridplot([[p1], [p2], [p3]])
    output_file("hist.html")
    show(p)

def plot_datatables(df_comp1, df_comp2, df_comp3):
    div1 = Div(text="""Position change""", width=200, height=100)
    p1_2 = DataTable(columns = [TableColumn(field=col, title=col) for col in df_comp1.columns], source = ColumnDataSource(df_comp1), height = 450, width = 400)

    div2 = Div(text="""Clicks change""", width=200, height=100)
    p2_2 = DataTable(columns = [TableColumn(field=col, title=col) for col in df_comp2.columns], source = ColumnDataSource(df_comp2), height = 450, width = 400)

    div3 = Div(text="""Impressions change""", width=200, height=100)
    p3_2 = DataTable(columns = [TableColumn(field=col, title=col) for col in df_comp3.columns], source = ColumnDataSource(df_comp3), height = 450, width = 400)

    p2 = gridplot([[div1, div2, div3], [p1_2, p2_2, p3_2]])
    # p = gridplot([[p1], [p2], [p3]])
    # output_file("hist.html")
    show(p2)

def print_summary(df_comp):
  print('')
  print('---------------------Summary--------------------')
  period_len = pd.to_datetime(end_date) - pd.to_datetime(start_date)
  print(f'Period length: {period_len}')
  all = len(df_comp)
  print(f'Total number of clients in the sample: {all}')
  pos_len = len(df_comp.loc[df_comp['Difference'] >= 0])
  print(f'Clients with positive change: {pos_len} ({"{:.2%}".format(pos_len/all)})')
  neg_len = len(df_comp.loc[df_comp['Difference'] < 0])
  print(f'Clients with negative change: {neg_len} ({"{:.2%}".format(neg_len/all)})')

  print('')
  print('---------------------Normal Distribution checks--------------------')
  mean_diff = df_comp['Difference'].mean()
  print(f'Mean difference: {"{:.2}".format(mean_diff)}')
  median_diff = df_comp['Difference'].median()
  print(f'Median difference: {"{:.2}".format(median_diff)}')
  skewness = df_comp['Difference'].skew()
  print(f'Skewness: {"{:.2}".format(skewness)} (Interpretation: -1: Skewed to the left, 0: Not skewed, 1: Skewed to the right)')
  kurt = df_comp['Difference'].kurtosis()
  print(f'Kurtosis: {"{:.2}".format(kurt)} (Interpretation: -1: Long tail(s), low middle point, 0: Standard bell shape, 1: short tail(s), high middle point)')

  one_std = len(df_comp.loc[(df_comp['Difference'] >= -df_comp['Difference'].std()) & (df_comp['Difference'] < df_comp['Difference'].std())])
  print(f'Data within 1 Standard Deviation from the mean: {one_std} ({"{:.2%}".format(one_std/all)})')
  two_std = len(df_comp.loc[(df_comp['Difference'] >= - 2*df_comp['Difference'].std()) & (df_comp['Difference'] < 2*df_comp['Difference'].std())])
  print(f'Data within 2 Standard Deviations from the mean: {two_std} ({"{:.2%}".format(two_std/all)})')
  three_std = len(df_comp.loc[(df_comp['Difference'] >= - 3*df_comp['Difference'].std()) & (df_comp['Difference'] < 3*df_comp['Difference'].std())])
  print(f'Data within 3 Standard Deviations from the mean: {three_std} ({"{:.2%}".format(three_std/all)})')

  print('')
  print('--------------------Outliers---------------------')
  first_q = len(df_comp.loc[df_comp['Difference'] <= -0.5])
  print(f'Clients with difference of less than -50%: {first_q} ({"{:.2%}".format(first_q/all)})')
  fourth_q = first_q = len(df_comp.loc[df_comp['Difference'] > 0.5])
  print(f'Clients with difference of more than 50%: {fourth_q} ({"{:.2%}".format(fourth_q/all)})')

# 3. Histograms
* Distribution of clients on a histogram based on their median change before and after.
* Having all clients on a single plot helps identify if a single big event (like google algorythm update) happened and in what way affected clients
* We can mark a select a single client to map where they lie on the histogram

# 3.1 Sessions

In [14]:
output_notebook()

import time

start_date = '2022-05-01' #@param {type: "date"}
end_date = '2022-06-30' #@param {type: "date"}
n_bins =  20#@param {type:"slider", min:10, max:30, step:1}
client = 'None'

df_comp, df_comp_slope = data_transform(df, start_date, end_date)


plot_hist(df_comp, n_bins, client)
print_summary(df_comp)
# print(df_comp.sort_values('Difference', ascending = True))

Loading BokehJS ...

# 3.2 Goal Completions

In [ ]:
output_notebook()

import time

start_date = '2022-05-26' #@param {type: "date"}
end_date = '2022-06-27' #@param {type: "date"}
n_bins =  20#@param {type:"slider", min:10, max:30, step:1}
client = 'None'

df_comp1, df_comp_slope1 = data_transform(df1, start_date, end_date)


plot_hist(df_comp1, n_bins, client)
print_summary(df_comp1)
# print(df_comp.sort_values('Difference', ascending = True))

# 4. Search console data

In [ ]:
output_notebook()

import time

start_date = '2022-05-26' #@param {type: "date"}
end_date = '2022-06-26' #@param {type: "date"}
n_bins =  25#@param {type:"slider", min:10, max:30, step:1}
client = 'None' 

df_comp1, df_comp_slope1 = data_transform(df1, start_date, end_date)
df_comp2, df_comp_slope2 = data_transform(df2, start_date, end_date)
df_comp3, df_comp_slope3 = data_transform(df3, start_date, end_date)

plot_grid(df_comp1, df_comp2, df_comp3, n_bins, client)
plot_datatables(df_comp1, df_comp2, df_comp3)
print_summary(df_comp1)
print_summary(df_comp2)
print_summary(df_comp3)